In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

df = pd.read_csv('/content/drive/MyDrive/빅리더/머신러닝 study/2017DC1-all.csv')
                              # 연속변수 타겟 VALP를 가지고 스케일 조정이 되지 않은 데이터셋인 house-unscaled-VALP.csv 불러오기
df.shape

In [ ]:
data = df.drop(['VALP'], axis=1)   # 타겟변수(VALP임에 주의)를 제외한 입력변수를 data에 저장
target = df['VALP']                # 타겟변수만 target에 저장

# 50:50 데이터 변환
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.5, random_state=42)   # test_size=0.5임에 주의

In [ ]:
# 기본 lGBMRegressor 모델
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import StratifiedKFold

lgb = LGBMRegressor(random_state=0)
lgb.fit(X_train, y_train)
pred = lgb.predict(X_test)

print('r2: {0:.5f}'.format(r2_score(y_test, pred)))

In [ ]:
#그리드 서치 실행
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

lgb = LGBMRegressor()

parameters = {'colsample_bytree': [0.9],  # 각각의 트리마다 사용할 컬럼의 비율
              'learning_rate': [0.1],
              'max_depth': [17], # 17까지 증가 이후 감소
              'min_child_weight' : [4], # 자식 노드의 갯수 증가 값이 거의 불변
              'n_estimators': [3000], # 실행될 부스팅 갯수
              'subsample': [0.21] # 낮춰야 함 각각의 스텝마다 사용할 샘플의 비율
              }

lgb_grid = GridSearchCV(lgb,
                        parameters,
                        scoring = 'r2',
                        cv = StratifiedKFold(n_splits=5, shuffle = True, random_state=42),
                        n_jobs = -1,
                        verbose=True)
lgb_grid.fit(X_train, y_train)


# parameters = {'colsample_bytree': [0.9],  # 높여야 함 각각의 트리마다 사용할 컬럼의 비율
#               'learning_rate': [0.1],
#               'max_depth': [17], # 17까지 증가 이후 감소
#               'min_child_weight' : [4], # 자식 노드의 갯수 증가 값이 거의 불변
#               'n_estimators': [1000], # 실행될 부스팅 갯수
#               'subsample': [0.21,0.2] # 낮춰야 함 각각의 스텝마다 사용할 샘플의 비율
#               } === 0.74976

In [ ]:
print('GridSearchCV 최적 파라미터:', lgb_grid.best_params_)

In [ ]:
from sklearn.metrics import r2_score

model = lgb_grid.best_estimator_
pred = model.predict(X_test)

print('r2: {0:.5f}'.format(r2_score(y_test, pred)))